In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

import seaborn as sns

%matplotlib inline

In [4]:
df = pd.read_csv('df_final.csv', index_col=0)

In [5]:
df = df[~df.duplicated()]

In [18]:
df.head()

,price,proc_cores,proc_speed,mem_cap,storage_cap,ssd,integrated,nvidia,ssdcap,corespeed**2,ddr4,ssdddr4,nvidia_ssd
0,930,6.0,3.2,16.0,2048.0,0,0,1,0.0,368.64,1,0,0
1,1050,6.0,3.2,16.0,2048.0,0,0,1,0.0,368.64,1,0,0
2,379,2.0,3.6,4.0,1024.0,0,1,0,0.0,51.84,1,0,0
3,674,4.0,3.1,8.0,256.0,1,1,0,256.0,153.76,1,1,0
4,823,6.0,3.1,16.0,1024.0,0,1,0,0.0,345.96,1,0,0


## Train test split

In [19]:
X, y = (df.iloc[:, 1:], df.iloc[:, 0])

X_train, X_test, y_train, y_test = train_test_split(X, y)

## Standardizing

In [20]:
ss = StandardScaler()

X_train, X_test = (ss.fit_transform(X_train), ss.fit_transform(X_test))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


## Modelling

### Ridge and Lasso

In [21]:
ridge = RidgeCV(cv=5)

ridge.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=5, fit_intercept=True,
    gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)

In [22]:
ridge.score(X_train, y_train)

0.7871956727192635

In [23]:
lasso = LassoCV(cv=3)

lasso.fit(X_train, y_train)

lasso.score(X_train, y_train)

0.7871890220941647

In [24]:
lasso.score(X_test, y_test)

0.7904171822930223

In [25]:
lasso.coef_

array([ 13.17173188,   4.59548743,  36.38997019,  17.05089878,
        28.95143173, -11.41135624,  15.66586824, -12.1035832 ,
        32.73928092,  89.65953944,  19.22119325,   5.49179121])

## OLS

In [26]:
lr = LinearRegression()

print(cross_val_score(lr, X_train, y_train, cv=5))

[0.77979329 0.78087324 0.74570298 0.70644313 0.81651022]


In [27]:
lr.fit(X_train, y_train)

lr.score(X_test, y_test)

0.7910366359601076

## Random forest regressor

In [28]:
rfr = RandomForestRegressor()

cross_val_score(rfr, X_train, y_train, cv=5)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The de

array([0.7272305 , 0.8356086 , 0.75217866, 0.81364379, 0.87574621])

In [36]:
rfr.fit(X_train, y_train)

tuning_parameters = {'n_estimators': [100, 400, 800], 'max_features': [4, 6, 8]}

gs = GridSearchCV(rfr, tuning_parameters, scoring='r2', n_jobs=-1, cv=5)

gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [100, 400, 800], 'max_features': [4, 6, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [37]:
gs.best_score_

0.8132465611516043

In [39]:
gs.score(X_train, y_train)

0.9341181139516247

In [38]:
gs.score(X_test, y_test)

0.822866465278416

In [40]:
gs.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=800, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

## Gradient Boosting Regressor

In [31]:
gbr = GradientBoostingRegressor()

cross_val_score(gbr, X_train, y_train, cv=5)

array([0.7858441 , 0.84992934, 0.77173187, 0.82140609, 0.86740073])

In [32]:
gbr.fit(X_train, y_train)

gbr.score(X_test, y_test)

0.8105676975993403